## ASSIGNMENT BY PRAKNAM S^3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
import math

# Function to Random shuffle the dataset into training and testing

In [2]:
def random_shuffle(values,ratio):
    random.shuffle(values)
    limit=int(math.ceil(len(values) * ratio))
    train = values[: limit]
    test = values[limit: ]
    x_train, y_train, x_test, y_test = [], [], [], []
    for i in train:
        x_train.append(i[: 6])
        y_train.append(i[6])
    for i in test:
        x_test.append(i[: 6])
        y_test.append(i[6])
        x_train = np.array([np.array(i) for i in x_train])
        
    ones=np.ones((x_train.shape[0],1))
    y_train = np.array([np.array(i) for i in y_train])
    x_test = np.array([np.array(i) for i in x_test])
    ones=np.ones((x_test.shape[0],1))
    y_test = np.array([np.array(i) for i in y_test])    
    return x_train, y_train, x_test, y_test

# Function to Standardize the dataset

In [3]:
def feature_scaling(lines):
    for i in range(6):
        tmp = []
        for line in lines:
            tmp.append(line[i])
        tmp = np.array(tmp)
        tmp_mean = tmp.mean()
        tmp_std = tmp.std(ddof = 0)
        for line in lines:            
            line[i] = (line[i] - tmp_mean) / tmp_std
            
    return lines

# Sigmoid Function 

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of Sigmoid Function

In [5]:
def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))


# Softmax Function

In [6]:
def softmax(A):
    expA = np.exp(A)
    return expA / expA.sum(axis=1, keepdims=True)

# Importing the dataset

In [7]:
data = pd.read_csv('dataset_NN.csv',header=0,delimiter=",")
print(data)

      att1  att2  att3        att4      att5         att6  class
0      1.0   0.0   0.0  133.150861  1.311693  1620.221779      1
1      1.0   0.0   0.0  126.724861  1.302745  1609.334822      1
2      1.0   0.0   0.0  131.173861  1.319031  1568.978435      1
3      1.0   0.0   0.0  129.478861  1.270878  1695.055281      1
4      1.0   0.0   0.0  127.262861  1.329637  1647.720235      1
...    ...   ...   ...         ...       ...          ...    ...
1995   1.0   1.0   1.0  157.498861  1.655794  5326.025889     10
1996   1.0   1.0   1.0  152.404861  1.620345  5243.267754     10
1997   1.0   1.0   1.0  134.672861  1.541987  3766.763222     10
1998   1.0   1.0   1.0  142.926861  1.426381  4118.327320     10
1999   1.0   1.0   1.0  133.920861  1.564621  3808.021317     10

[2000 rows x 7 columns]


In [8]:
att1 = list(data['att1'])
att2 = list(data['att2'])
att3 = list(data['att3'])
att4 = list(data['att4'])
att5 = list(data['att5'])
att6 = list(data['att6'])
actual = list(data['class'])
values = []

for i in range(len(actual)):
    values.append([att1[i], att2[i], att3[i], att4[i],att5[i],att6[i]])
    
values = feature_scaling(values)

for i in range(len(actual)):
    values[i].append(actual[i])

x_train, y_train, x_test, y_test=random_shuffle(values,0.7)

# Specifying the learning rate

In [9]:
lr = 10e-4

# Feed Forward

In [10]:
def calculate_accuracy(x,wh,wo,bh,bo):
    
    # Phase 1
    zh = np.dot(x, weight) + bias
    ah = sigmoid(zh)

    # Phase 2
    zo = np.dot(ah, wo) + bo
    ao = softmax(zo)
    
    # print(ao)
    ao_list = ao.tolist()
    predicted = []
    for i in range(len(y_train)):
        max_pos = ao_list[i].index(max(ao_list[i]))
        predicted.append(max_pos+1)
    count = 0;
    for i in range(len(y_train)):
        if(predicted[i]==y_train[i]):
            count=count+1
    print("Training Accuracy=",count/len(y_train)*100)

# Main Function 

In [11]:
input_nodes = 6
output_nodes = 10

hidden_nodes = 4
output_labels = 10


wh = np.random.rand(input_nodes,hidden_nodes)
bh = np.random.randn(hidden_nodes)

wo = np.random.rand(hidden_nodes,output_labels)
bo = np.random.randn(output_labels)
error_cost = []
one_hot_labels = np.zeros((len(y_train), output_labels))
for i in range(len(y_train)):
    index = y_train[i];
    one_hot_labels[i][index-1]=1
    
for epoch in range(50000):
    
    ############# feedforward

    # Phase 1
    zh = np.dot(x_train, wh) + bh
    ah = sigmoid(zh)

    # Phase 2
    zo = np.dot(ah, wo) + bo
    ao = softmax(zo)


    
    ########## Back Propagation

########## Phase 1
    dcost_dzo = ao - one_hot_labels # d_cost/d_ao * d_ao/d_zo
    dzo_dwo = ah 
    dcost_wo = np.dot(dzo_dwo.T, dcost_dzo)
    dcost_bo = dcost_dzo

########## Phases 2
    dzo_dah = wo
    dcost_dah = np.dot(dcost_dzo , dzo_dah.T)
    dah_dzh = sigmoid_der(zh)
    dzh_dwh = x_train
    dcost_wh = np.dot(dzh_dwh.T, dah_dzh * dcost_dah)
    dcost_bh = dcost_dah * dah_dzh

    # Update Weights ================

    wh -= lr * dcost_wh
    bh -= lr * dcost_bh.sum(axis=0)

    wo -= lr * dcost_wo
    bo -= lr * dcost_bo.sum(axis=0)

    if epoch % 200 == 0:
        loss = np.sum(-one_hot_labels * np.log(ao))
        print('Loss function value: ', loss)
        error_cost.append(loss)

# Phase 1
zh = np.dot(x_train, wh) + bh
ah = sigmoid(zh)

# Phase 2
zo = np.dot(ah, wo) + bo
ao = softmax(zo)
# print(ao)
ao_list = ao.tolist()
predicted = []
for i in range(len(y_train)):
    max_pos = ao_list[i].index(max(ao_list[i]))
    predicted.append(max_pos+1)
count = 0;
for i in range(len(y_train)):
    if(predicted[i]==y_train[i]):
        count=count+1
print("Training Accuracy=",count/len(y_train)*100)


# TESTING ACCURACY
testing_a = []
# Phase 1
zh = np.dot(x_test, wh) + bh
ah = sigmoid(zh)

# Phase 2
zo = np.dot(ah, wo) + bo
ao = softmax(zo)
# print(ao)
ao_list = ao.tolist()     
for i in range(len(y_test)):
    max_pos = ao_list[i].index(max(ao_list[i]))
    testing_a.append(max_pos+1)


count = 0;    
# print(len(testing_a))
# print(testing_a)
for i in range(len(y_test)):
    if(testing_a[i]==y_test[i]):
        count=count+1
print("Testing Accuracy=",count/len(y_test)*100)  

Loss function value:  3274.0613385596826
Loss function value:  1218.9982947996373
Loss function value:  1036.5267366585017
Loss function value:  965.3133108774032
Loss function value:  918.5819451732386
Loss function value:  888.1345749639702
Loss function value:  870.6860672126567
Loss function value:  859.9550045491432
Loss function value:  852.4236007485663
Loss function value:  846.6120463783293
Loss function value:  841.8598414910366
Loss function value:  837.8247158104796
Loss function value:  834.3065901648188
Loss function value:  831.19116095765
Loss function value:  828.4221826242737
Loss function value:  825.963506046228
Loss function value:  823.7756054113145
Loss function value:  821.8171359785474
Loss function value:  820.0508717464504
Loss function value:  818.4458224815664
Loss function value:  816.9769448412469
Loss function value:  815.6241387916284
Loss function value:  814.3712163319492
Loss function value:  813.2050344293314
Loss function value:  812.1148143566887


Loss function value:  788.3208479610277
Loss function value:  788.3024063911316
Loss function value:  788.2841925120795
Loss function value:  788.266206364339
Loss function value:  788.248447963719
Loss function value:  788.230917297154
Loss function value:  788.2136143185572
Loss function value:  788.1965389447278
Loss function value:  788.179691051346
Loss function value:  788.1630704690475
Loss function value:  788.1466769795894
Loss function value:  788.1305103121333
Loss function value:  788.1145701396412
Loss function value:  788.0988560753985
Loss function value:  788.0833676696768
Loss function value:  788.0681044065446
Loss function value:  788.0530657008476
Loss function value:  788.0382508953404
Loss function value:  788.0236592580193
Loss function value:  788.0092899796309
Loss function value:  787.995142171387
Loss function value:  787.9812148628815
Loss function value:  787.9675070002315
Loss function value:  787.9540174444344
Loss function value:  787.9407449699554
Loss 